In [1]:
pip install tensorflow keras numpy requests scikit-learn matplotlib ConfigSpace

  Using cached configspace-1.2.0.tar.gz (130 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached more_itertools-10.5.0-py3-none-any.whl.metadata (36 kB)
Using cached more_itertools-10.5.0-py3-none-any.whl (60 kB)
  Created wheel for ConfigSpace: filename=ConfigSpace-1.2.0-py3-none-any.whl size=115855 sha256=3ea14f70bb63c1e378e671ef23dabf63fce228c03be34680bca9cf0c7fc69a55
  Stored in directory: /vol/home/s4422090/.cache/pip/wheels/e1/d9/f5/da696a970b97ddcd69be38c41ca81ab5d6634234b65d870d98
Successfully built ConfigSpace
Note: you may need to restart the kernel to use updated packages.


In [3]:
import tensorflow as tf

gpus = tf.config.list_physical_devices('GPU')
print("Num GPUs Available:", len(gpus))

if gpus:
    try:
        # Enable memory growth for each GPU to prevent TensorFlow from allocating all GPU memory at once
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        print("Error setting memory growth:", e)
else:
    print("No GPU detected. Using CPU.")

Num GPUs Available: 0
No GPU detected. Using CPU.


## Perform Grid Search on Fashion MNIST dataset

In [ ]:
import numpy as np
import tensorflow as tf
from keras.datasets import fashion_mnist
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from sklearn.model_selection import train_test_split
from ConfigSpace import ConfigurationSpace
from ConfigSpace.hyperparameters import CategoricalHyperparameter, Constant
from ConfigSpace.util import generate_grid


def train_model(config):
    # Extract hyperparameters from the configuration
    filters = config['filters']
    kernel_size = config['kernel_size']
    activation = config['activation']
    optimizer = config['optimizer']
    batch_size = config['batch_size']
    epochs = config['epochs']
    kernel_initializer = config['kernel_initializer']
    dropout_rate = config['dropout_rate']
    max_pool_size = config['max_pool_size']
    FC_size = config['FC_size']
    

    # Build the model
    model = Sequential([
        Conv2D(filters, kernel_size=(kernel_size, kernel_size), activation=activation, input_shape=input_shape, kernel_initializer=kernel_initializer),
        Conv2D(filters * 2, kernel_size=(kernel_size, kernel_size), activation=activation, kernel_initializer=kernel_initializer),
        MaxPooling2D(pool_size=(max_pool_size, max_pool_size)),
        Dropout(dropout_rate),

        Flatten(),
        Dense(FC_size, activation=activation, kernel_initializer=kernel_initializer),
        Dropout(0.5),
        Dense(num_classes, activation='softmax')
    ])

    # Compile the model
    model.compile(
        loss='categorical_crossentropy',
        optimizer=optimizer,
        metrics=['accuracy']
    )

    # Train the model
    history = model.fit(
        X_train, y_train,
        batch_size=batch_size,
        epochs=epochs,
        verbose=0,  
        validation_data=(X_valid, y_valid)
    )

    # Evaluate the model on the validation set
    val_loss, val_accuracy = model.evaluate(X_valid, y_valid, verbose=0)

    return val_accuracy


# Load the Fashion MNIST dataset
(X_train_full, y_train_full), (X_test, y_test) = fashion_mnist.load_data()

# Normalize the input data to [0, 1] range
X_train_full = X_train_full.astype('float32') / 255.0
X_test = X_test.astype('float32') / 255.0

# Split the training set into training and validation sets
val_ratio = 0.1
X_train, X_valid, y_train, y_valid = train_test_split(
    X_train_full, y_train_full, test_size=val_ratio, random_state=42, shuffle=True
)

# Reshape input data
img_rows, img_cols = 28, 28
X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols, 1)
X_valid = X_valid.reshape(X_valid.shape[0], img_rows, img_cols, 1)
X_test = X_test.reshape(X_test.shape[0], img_rows, img_cols, 1)
input_shape = (img_rows, img_cols, 1)

# One-hot encoding of target labels
num_classes = 10
y_train = to_categorical(y_train, num_classes)
y_valid = to_categorical(y_valid, num_classes)
y_test = to_categorical(y_test, num_classes)


# Define the hyperparameter space
cs = ConfigurationSpace()

# Hyperparameters
kernel_initializer = CategoricalHyperparameter('kernel_initializer', ['random_normal', 'he_normal'])
activation = CategoricalHyperparameter('activation', ['relu', 'sigmoid'])
optimizer = CategoricalHyperparameter('optimizer', ['adam', 'sgd'])
filters = CategoricalHyperparameter('filters', [28, 56])
kernel_size = CategoricalHyperparameter('kernel_size', [3, 5])
dropout_rate = CategoricalHyperparameter('dropout_rate', [0.0, 0.25, 0.5])
max_pool_size = CategoricalHyperparameter('max_pool_size', [2,3])
FC_size = CategoricalHyperparameter('FC_size', [128, 256])
batch_size = Constant('batch_size', 128)
epochs = Constant('epochs', 10)

# Add hyperparameters to the configuration space
cs.add_hyperparameters([filters, kernel_size, activation, optimizer, batch_size, epochs, kernel_initializer, dropout_rate, max_pool_size, FC_size])

# Generate the grid of configurations
grid = generate_grid(cs)
print(f"Total configurations to evaluate: {len(grid)}")

results = []

for idx, config in enumerate(grid):
    config_dict = config.get_dictionary()
    print(f"Evaluating configuration {idx + 1}/{len(grid)}: {config_dict}")
    val_accuracy = train_model(config_dict)
    print(f"Validation accuracy: {val_accuracy:.4f}\n")

    # Store the results
    results.append({
        'config': config_dict,
        'val_accuracy': val_accuracy
    })

# Sort the results by validation accuracy in descending order
results = sorted(results, key=lambda x: x['val_accuracy'], reverse=True)


## Analyze the Grid Search results

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from scipy import stats
import statsmodels.api as sm
from statsmodels.formula.api import ols


# Create a DataFrame from the results
df_results = pd.DataFrame(results)
config_df = pd.json_normalize(df_results['config'])
df = pd.concat([config_df, df_results['val_accuracy']], axis=1)

# Display the first few rows
print("First first 10 rows of the DataFrame:")
display(df.head(10))


# ANOVA Analysis

df_anova = df.copy()

# List of hyperparameters to include in the ANOVA table
anova_hps = ['filters', 'kernel_size', 'kernel_initializer', 'activation', 'optimizer', 'dropout_rate', 'max_pool_size', 'FC_size']



CAN I DELETE THIS? 
# Convert hyperparameters to categorical data types
for col in anova_hps:
    df_anova[col] = df_anova[col].astype('category')

# Construct the formula for ANOVA
formula = 'val_accuracy ~ ' + ' + '.join(anova_hps)

# Perform ANOVA
model = ols(formula, data=df_anova).fit()
anova_table = sm.stats.anova_lm(model, typ=2)

# Round the 'PR(>F)' column to four decimal places
anova_table['PR(>F)'] = anova_table['PR(>F)'].round(4)

# Display the ANOVA table
print("ANOVA Table:")
display(anova_table)


# Average Validation Accuracy for Each Hyperparameter Value

# List of hyperparameters to include in the table 
hyperparameters = ['filters', 'kernel_size', 'kernel_initializer', 'activation', 'optimizer', 'dropout_rate', 'max_pool_size', 'FC_size']


mean_accuracy_list = []

# Loop through each hyperparameter to calculate mean validation accuracy per value
for hp in hyperparameters:
    # Group by the hyperparameter and calculate the mean validation accuracy
    hp_mean = df.groupby(hp)['val_accuracy'].mean().reset_index()
    
    # Rename the 'val_accuracy' column to 'Mean Val Accuracy'
    hp_mean = hp_mean.rename(columns={'val_accuracy': 'Mean Val Accuracy', hp: 'Value'})
    
    # Add a column to indicate which hyperparameter the row corresponds to
    hp_mean.insert(0, 'Hyperparameter', hp.replace('_', ' ').capitalize())
    
    # Append the result to the list
    mean_accuracy_list.append(hp_mean)

# Concatenate all the individual hyperparameter DataFrames into one
mean_accuracy_df = pd.concat(mean_accuracy_list, ignore_index=True)

# Round the mean validation accuracy to four decimals
mean_accuracy_df['Mean Val Accuracy'] = mean_accuracy_df['Mean Val Accuracy'].round(4)

# Display the table
print("Average Validation Accuracy for Each Hyperparameter Value:")
display(mean_accuracy_df)

In [ ]:
import numpy as np
import tensorflow as tf
from keras.datasets import fashion_mnist
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from sklearn.model_selection import train_test_split

# Load and preprocess data
(X_train_full, y_train_full), (X_test, y_test) = fashion_mnist.load_data()

X_train_full = X_train_full.astype('float32') / 255.0
X_test = X_test.astype('float32') / 255.0

val_ratio = 0.1
X_train, X_valid, y_train, y_valid = train_test_split(
    X_train_full, y_train_full, test_size=val_ratio, random_state=42, shuffle=True
)

img_rows, img_cols = 28, 28
X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols, 1)
X_valid = X_valid.reshape(X_valid.shape[0], img_rows, img_cols, 1)
X_test = X_test.reshape(X_test.shape[0], img_rows, img_cols, 1)
input_shape = (img_rows, img_cols, 1)

num_classes = 10
y_train = to_categorical(y_train, num_classes)
y_valid = to_categorical(y_valid, num_classes)
y_test = to_categorical(y_test, num_classes)


# Function to build the model with a given number of blocks
def build_model(num_blocks):
    model = Sequential()
    
    # First Convolutional Layer
    model.add(Conv2D(28, kernel_size=(3, 3), activation='relu', input_shape=input_shape, kernel_initializer='he_normal'))
    model.add(Conv2D(28, kernel_size=(3, 3), activation='relu', kernel_initializer='he_normal'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))


    for i in range(num_blocks):
        filters = 28 * (2 ** i)

        model.add(Conv2D(filters, kernel_size=(3, 3), activation='relu', kernel_initializer='he_normal'))
        model.add(Conv2D(filters, kernel_size=(3, 3), activation='relu', kernel_initializer='he_normal'))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Dropout(0.25))       

    model.add(Flatten())
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='softmax'))

    model.compile(
        loss='categorical_crossentropy',
        optimizer='adam',
        metrics=['accuracy']
    )
    return model

# Loop through the number of blocks and evaluate the model
validation_accuracies = []

for num_blocks in range(1, 4):
    print(f"Training model with {num_blocks} block(s)...")
    model = build_model(num_blocks)
    model.fit(
        X_train, y_train,
        batch_size=128,
        epochs=10,
        verbose=0,
        validation_data=(X_valid, y_valid)
    )
    score = model.evaluate(X_valid, y_valid, verbose=0)
    validation_accuracies.append(score[1])
    print(f"Validation accuracy with {num_blocks} block(s): {score[1]:.4f}")

# Print the validation accuracies for each number of blocks
for num_blocks, val_acc in enumerate(validation_accuracies, start=1):
    print(f"Validation accuracy with {num_blocks} block(s): {val_acc:.4f}")

In [ ]:
import numpy as np
import tensorflow as tf
from keras.datasets import fashion_mnist
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from sklearn.model_selection import train_test_split

# Load and preprocess data
(X_train_full, y_train_full), (X_test, y_test) = fashion_mnist.load_data()

X_train_full = X_train_full.astype('float32') / 255.0
X_test = X_test.astype('float32') / 255.0

val_ratio = 0.1
X_train, X_valid, y_train, y_valid = train_test_split(
    X_train_full, y_train_full, test_size=val_ratio, random_state=42, shuffle=True
)

img_rows, img_cols = 28, 28
X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols, 1)
X_valid = X_valid.reshape(X_valid.shape[0], img_rows, img_cols, 1)
X_test = X_test.reshape(X_test.shape[0], img_rows, img_cols, 1)
input_shape = (img_rows, img_cols, 1)

num_classes = 10
y_train = to_categorical(y_train, num_classes)
y_valid = to_categorical(y_valid, num_classes)
y_test = to_categorical(y_test, num_classes)


# Function to build the model with a given number of blocks
def build_model(num_blocks):
    model = Sequential()
    
    # First Convolutional Layer
    model.add(Conv2D(28, kernel_size=(3, 3), activation='relu', input_shape=input_shape, kernel_initializer='he_normal'))
    model.add(Conv2D(28, kernel_size=(3, 3), activation='relu', kernel_initializer='he_normal'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))


    for i in range(num_blocks):
        filters = 28 * (2 ** i)

        model.add(Conv2D(filters, kernel_size=(3, 3), activation='relu', kernel_initializer='he_normal'))
        model.add(Conv2D(filters, kernel_size=(3, 3), activation='relu', kernel_initializer='he_normal'))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Dropout(0.25))       

    model.add(Flatten())
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='softmax'))

    model.compile(
        loss='categorical_crossentropy',
        optimizer='adam',
        metrics=['accuracy']
    )
    return model

# Loop through the number of blocks and evaluate the model
validation_accuracies = []

for num_blocks in range(1, 4):
    print(f"Training model with {num_blocks} block(s)...")
    model = build_model(num_blocks)
    model.fit(
        X_train, y_train,
        batch_size=128,
        epochs=10,
        verbose=0,
        validation_data=(X_valid, y_valid)
    )
    score = model.evaluate(X_valid, y_valid, verbose=0)
    validation_accuracies.append(score[1])
    print(f"Validation accuracy with {num_blocks} block(s): {score[1]:.4f}")

# Print the validation accuracies for each number of blocks
for num_blocks, val_acc in enumerate(validation_accuracies, start=1):
    print(f"Validation accuracy with {num_blocks} block(s): {val_acc:.4f}")

## Test 100 epochs on best Config, best number of conv blocks

## CIFAR 3 - Top 3 best configurations

In [ ]:
from __future__ import print_function
import keras
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras import backend as K
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import tensorflow as tf



(X_train_full, y_train_full), (X_test, y_test) = cifar10.load_data()

X_train_full = X_train_full.astype('float32') / 255.0
X_test = X_test.astype('float32') / 255.0

val_ratio = 0.1
X_train, X_valid, y_train, y_valid = train_test_split(
    X_train_full, y_train_full, test_size=val_ratio, random_state=42, shuffle=True
)

# Define class names for reference 
class_names = [
    'airplane', 'automobile', 'bird', 'cat', 'deer',
    'dog', 'frog', 'horse', 'ship', 'truck'
]

#  Print dataset shapes 
print('X_train shape:', X_train.shape)
print(X_train.shape[0], 'train samples')
print(X_valid.shape[0], 'validation samples')
print(X_test.shape[0], 'test samples')

# Set input shape
img_rows, img_cols, img_channels = 32, 32, 3
input_shape = (img_rows, img_cols, img_channels)

# Flatten label arrays
y_train = y_train.flatten()
y_valid = y_valid.flatten()
y_test = y_test.flatten()

# One-hot encoding
num_classes = 10  

y_train = to_categorical(y_train, num_classes)
y_valid = to_categorical(y_valid, num_classes)
y_test = to_categorical(y_test, num_classes)

kernel_initializer = 'he_normal'

# 9. Define the CNN model
model = Sequential([
    Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same', input_shape=input_shape, kernel_initializer=kernel_initializer),
    Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same', kernel_initializer=kernel_initializer),
    MaxPooling2D(pool_size=(3, 3)),
    Dropout(0.25),

    Flatten(),
    Dense(512, activation='relu', kernel_initializer=kernel_initializer),
    Dropout(0.5),
    Dense(num_classes, activation='softmax')
])

# 10. Compile the model
model.compile(
    loss='categorical_crossentropy',      # Suitable loss function for multi-class classification
    optimizer='adam',                     # Adam optimizer
    metrics=['accuracy']                  # Evaluate performance using accuracy
)

# 11. Train the model
model.fit(
    X_train, y_train,
    batch_size=128,
    epochs=10,
    verbose=0,
    validation_data=(X_valid, y_valid)    # Use validation set for monitoring
)

# 12. Evaluate the model on the test set
score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

# config 2

kernel_initializer = 'random_normal'

# 9. Define the CNN model
model = Sequential([
    Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same', input_shape=input_shape, kernel_initializer=kernel_initializer),
    Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same', kernel_initializer=kernel_initializer),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.5),

    Flatten(),
    Dense(512, activation='relu', kernel_initializer=kernel_initializer),
    Dropout(0.5),
    Dense(num_classes, activation='softmax')
])

# 10. Compile the model
model.compile(
    loss='categorical_crossentropy',      # Suitable loss function for multi-class classification
    optimizer='adam',                     # Adam optimizer
    metrics=['accuracy']                  # Evaluate performance using accuracy
)

# 11. Train the model
model.fit(
    X_train, y_train,
    batch_size=128,
    epochs=10,
    verbose=0,
    validation_data=(X_valid, y_valid)    # Use validation set for monitoring
)

# 12. Evaluate the model on the test set
score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])


# config 3

kernel_initializer = 'random_normal'

model = Sequential([
    Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same', input_shape=input_shape, kernel_initializer=kernel_initializer),
    Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same', kernel_initializer=kernel_initializer),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.0),

    Flatten(),
    Dense(512, activation='relu', kernel_initializer=kernel_initializer),
    Dropout(0.5),
    Dense(num_classes, activation='softmax')
])

model.compile(
    loss='categorical_crossentropy',     
    optimizer='adam',                     
    metrics=['accuracy']                  
)

model.fit(
    X_train, y_train,
    batch_size=128,
    epochs=10,
    verbose=0,
    validation_data=(X_valid, y_valid)    
)

# 12. Evaluate the model on the test set
score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])




## CIFAR - test number of blocks

In [ ]:
from __future__ import print_function
import keras
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras import backend as K
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import tensorflow as tf

# Verify TensorFlow installation and GPU availability
print("TensorFlow version:", tf.__version__)

gpus = tf.config.list_physical_devices('GPU')
print("Num GPUs Available:", len(gpus))

if gpus:
    try:
        # Enable memory growth for each GPU to prevent TensorFlow from allocating all GPU memory at once
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        print("Error setting memory growth:", e)
else:
    print("No GPU detected. Using CPU.")

# 1. Load the CIFAR-10 dataset
(X_train_full, y_train_full), (X_test, y_test) = cifar10.load_data()

# 2. Normalize the input data to [0, 1] range
X_train_full = X_train_full.astype('float32') / 255.0
X_test = X_test.astype('float32') / 255.0

# 3. Split the full training set into training and validation sets
val_ratio = 0.1
X_train, X_valid, y_train, y_valid = train_test_split(
    X_train_full, y_train_full, test_size=val_ratio, random_state=42, shuffle=True
)

# 4. Define class names for reference (optional)
class_names = [
    'airplane', 'automobile', 'bird', 'cat', 'deer',
    'dog', 'frog', 'horse', 'ship', 'truck'
]

# 5. Print dataset shapes for verification
print('X_train shape:', X_train.shape)
print(X_train.shape[0], 'train samples')
print(X_valid.shape[0], 'validation samples')
print(X_test.shape[0], 'test samples')

# 6. Set input shape based on CIFAR-10 images
img_rows, img_cols, img_channels = 32, 32, 3
input_shape = (img_rows, img_cols, img_channels)

# 7. Flatten label arrays
y_train = y_train.flatten()
y_valid = y_valid.flatten()
y_test = y_test.flatten()

# 8. Convert class vectors to binary class matrices (one-hot encoding)
num_classes = 10  # There are 10 classes in CIFAR-10

y_train = to_categorical(y_train, num_classes)
y_valid = to_categorical(y_valid, num_classes)
y_test = to_categorical(y_test, num_classes)

kernel_initializer = 'he_normal'

# 9. Define the CNN model
def build_model(num_blocks):
    model = Sequential()

    model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', padding='same', input_shape=input_shape, kernel_initializer=kernel_initializer))
    model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', padding='same', kernel_initializer=kernel_initializer))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    for i in range(1, num_blocks):
        filters = 32 * (2 ** i)
        model.add(Conv2D(filters, kernel_size=(3, 3), activation='relu', padding='same', kernel_initializer=kernel_initializer))
        model.add(Conv2D(filters, kernel_size=(3, 3), activation='relu', padding='same', kernel_initializer=kernel_initializer))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Dropout(0.25))


    model.add(Flatten())
    model.add(Dense(512, activation='relu', kernel_initializer=kernel_initializer))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='softmax'))

    model.compile(
        loss='categorical_crossentropy',    
        optimizer='adam',                     
        metrics=['accuracy']                 
    )
    return model

validation_accuracies = []

for num_blocks in range(1, 5):
    print(f"Training model with {num_blocks} block(s)...")
    model = build_model(num_blocks)
    model.fit(
        X_train, y_train,
        batch_size=128,
        epochs=10,
        verbose=0,
        validation_data=(X_valid, y_valid)
    )
    score = model.evaluate(X_valid, y_valid, verbose=0)
    validation_accuracies.append(score[1])
    print(f"Validation accuracy with {num_blocks} block(s): {score[1]:.4f}")

# Print the validation accuracies for each number of blocks
for num_blocks, val_acc in enumerate(validation_accuracies, start=1):
    print(f"Validation accuracy with {num_blocks} block(s): {val_acc:.4f}")



## CIFAR - 100 epochs

In [ ]:
from __future__ import print_function
import keras
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import tensorflow as tf
import matplotlib.pyplot as plt  # Import for plotting

# Verify TensorFlow installation and GPU availability
print("TensorFlow version:", tf.__version__)

gpus = tf.config.list_physical_devices('GPU')
print("Num GPUs Available:", len(gpus))

if gpus:
    try:
        # Enable memory growth for each GPU to prevent TensorFlow from allocating all GPU memory at once
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        print("Error setting memory growth:", e)
else:
    print("No GPU detected. Using CPU.")

# 1. Load the CIFAR-10 dataset
(X_train_full, y_train_full), (X_test, y_test) = cifar10.load_data()

# 2. Normalize the input data to [0, 1] range
X_train_full = X_train_full.astype('float32') / 255.0
X_test = X_test.astype('float32') / 255.0

# 3. Split the full training set into training and validation sets
val_ratio = 0.1
X_train, X_valid, y_train, y_valid = train_test_split(
    X_train_full, y_train_full, test_size=val_ratio, random_state=42, shuffle=True
)

# 4. Define class names for reference (optional)
class_names = [
    'airplane', 'automobile', 'bird', 'cat', 'deer',
    'dog', 'frog', 'horse', 'ship', 'truck'
]

# 5. Print dataset shapes for verification
print('X_train shape:', X_train.shape)
print(X_train.shape[0], 'train samples')
print(X_valid.shape[0], 'validation samples')
print(X_test.shape[0], 'test samples')

# 6. Set input shape based on CIFAR-10 images
img_rows, img_cols, img_channels = 32, 32, 3
input_shape = (img_rows, img_cols, img_channels)

# 7. Flatten label arrays
y_train = y_train.flatten()
y_valid = y_valid.flatten()
y_test = y_test.flatten()

# 8. Convert class vectors to binary class matrices (one-hot encoding)
num_classes = 10  # There are 10 classes in CIFAR-10

y_train = to_categorical(y_train, num_classes)
y_valid = to_categorical(y_valid, num_classes)
y_test = to_categorical(y_test, num_classes)

kernel_initializer = 'he_normal'

# 9. Define the CNN model
model = Sequential([
    Conv2D(32, kernel_size=(3, 3), activation='relu', padding='same', input_shape=input_shape, kernel_initializer=kernel_initializer),
    Conv2D(32, kernel_size=(3, 3), activation='relu', padding='same', kernel_initializer=kernel_initializer),
    MaxPooling2D(pool_size=(3, 3)),
    Dropout(0.25),


    Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same', kernel_initializer=kernel_initializer),
    Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same', kernel_initializer=kernel_initializer),
    MaxPooling2D(pool_size=(3, 3)),
    Dropout(0.25),

    Flatten(),
    Dense(512, activation='relu', kernel_initializer=kernel_initializer),
    Dropout(0.5),
    Dense(num_classes, activation='softmax')
])

# 10. Compile the model
model.compile(
    loss='categorical_crossentropy',      # Suitable loss function for multi-class classification
    optimizer='adam',                     # Adam optimizer
    metrics=['accuracy']                  # Evaluate performance using accuracy
)

# 11. Train the model and capture the history
history = model.fit(
    X_train, y_train,
    batch_size=128,
    epochs=100,
    verbose=1,
    validation_data=(X_valid, y_valid)    # Use validation set for monitoring
)

# 12. Evaluate the model on the test set
score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

# 13. Plot training & validation accuracy values
import matplotlib.pyplot as plt

plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy over 100 Epochs')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()